In [1]:
from preprocessing import get_model_dataset, create_train_test, min_max_scale, df_to_xy, read_file, lag_features
from lstm import create_model
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [2]:
first_year = 2019
last_year = 2021
file = f"./data/processed_data/{first_year}-{last_year}.csv"

df_read = read_file(file)
print(df_read)
df_read.info()
print(df_read)
print(df_read["Ttl"].max())

         Unnamed: 0  Quote_date Expire_date  Underlying_last  Strike      Ask  \
0           1354913  2019-01-02  2019-01-04          2509.98   800.0  1711.10   
1           1354914  2019-01-02  2019-01-04          2509.98   900.0  1611.40   
2           1354915  2019-01-02  2019-01-04          2509.98  1000.0  1511.40   
3           1354916  2019-01-02  2019-01-04          2509.98  1050.0  1462.19   
4           1354917  2019-01-02  2019-01-04          2509.98  1100.0  1412.20   
...             ...         ...         ...              ...     ...      ...   
5128605     6526904  2021-12-31  2026-12-18          4766.39  8400.0    71.70   
5128606     6526905  2021-12-31  2026-12-18          4766.39  8600.0    63.60   
5128607     6526906  2021-12-31  2026-12-18          4766.39  8800.0    55.70   
5128608     6526907  2021-12-31  2026-12-18          4766.39  9000.0   119.80   
5128609     6526908  2021-12-31  2026-12-18          4766.39  9200.0    41.20   

             Bid  Bid_strik

In [7]:
tests = [10000, 1586, 1089, 365*2, 365, 180, 90, 60, 30, 15, 0]
for test in tests:
    print('Count Ttl >', test, ':', df_read["Ttl"][df_read['Ttl'] > test].count())

Count Ttl > 10000 : 0
Count Ttl > 1586 : 756
Count Ttl > 1089 : 5220
Count Ttl > 730 : 93665
Count Ttl > 365 : 367547
Count Ttl > 180 : 905013
Count Ttl > 90 : 1838838
Count Ttl > 60 : 2265523
Count Ttl > 30 : 3127822
Count Ttl > 15 : 4042733
Count Ttl > 0 : 5128610


In [8]:
tests = [50, 25, 10, 5, 2, 1.5, 1.03, 1, 0.97, 0.5, 0]
print(df_read["Moneyness"].max())
print(df_read["Moneyness"].min())
for test in tests:
    print('Moneyness Ttl >', test, ':', df_read["Moneyness"][df_read['Moneyness'] > test].count() / df_read["Moneyness"].count() * 100)

47.9294
0.4562775510204082
Moneyness Ttl > 50 : 0.0
Moneyness Ttl > 25 : 0.07216380266777939
Moneyness Ttl > 10 : 0.28793376762904566
Moneyness Ttl > 5 : 0.6801453025283654
Moneyness Ttl > 2 : 6.348484287165529
Moneyness Ttl > 1.5 : 13.907998463521304
Moneyness Ttl > 1.03 : 61.680357835748865
Moneyness Ttl > 1 : 69.05132189813614
Moneyness Ttl > 0.97 : 76.72375165980647
Moneyness Ttl > 0.5 : 99.9983621293099
Moneyness Ttl > 0 : 100.0


In [15]:
print(df_read["Strike"][df_read["Ttl"] <= 3*365].max())
print(df_read["Strike"].min())
tester = "Strike"
tests = [8600, 7600, 6600, 5600, 4600, 3600, 2600, 1600, 600, 100, 0]
for test in tests:
    print(tester, '>', test, ':', df_read[tester][df_read[tester] > test].count() / df_read[tester].count() * 100)

9200.0
100.0
Strike > 8600 : 0.005206088979275086
Strike > 7600 : 0.014818830053367288
Strike > 6600 : 0.2914044936152291
Strike > 5600 : 1.5009524998001407
Strike > 4600 : 7.644975929150394
Strike > 3600 : 30.260752913557475
Strike > 2600 : 73.20872517114773
Strike > 1600 : 94.19638069574407
Strike > 600 : 99.42274807403956
Strike > 100 : 99.92651030201165
Strike > 0 : 100.0


In [61]:
df_read[(df_read["Strike"] == 9200)]

,Unnamed: 0,Quote_date,Expire_date,Underlying_last,Strike,Ask,Bid,Bid_strike,Ask_strike,Moneyness,Ttl,Volatility,R
4784011,6172681,2021-11-05,2024-12-20,4697.16,9200.0,300.0,0.0,0.000000,0.032609,0.510561,1141,0.110251,0.66
4784063,6172733,2021-11-05,2025-12-19,4697.16,9200.0,300.0,0.0,0.000000,0.032609,0.510561,1505,0.110251,1.04
4784088,6172758,2021-11-05,2026-12-18,4697.16,9200.0,309.0,9.0,0.000978,0.033587,0.510561,1869,0.110251,1.04
4793475,6182145,2021-11-08,2024-12-20,4701.65,9200.0,19.2,0.0,0.000000,0.002087,0.511049,1138,0.109971,0.75
4793527,6182197,2021-11-08,2025-12-19,4701.65,9200.0,85.0,5.0,0.000543,0.009239,0.511049,1502,0.109971,1.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5119101,6517400,2021-12-30,2025-12-19,4779.28,9200.0,86.0,6.0,0.000652,0.009348,0.519487,1450,0.137746,0.98
5119161,6517460,2021-12-30,2026-12-18,4779.28,9200.0,43.1,19.1,0.002076,0.004685,0.519487,1814,0.137746,1.27
5128489,6526788,2021-12-31,2024-12-20,4766.39,9200.0,300.0,0.0,0.000000,0.032609,0.518086,1085,0.136456,0.97
5128549,6526848,2021-12-31,2025-12-19,4766.39,9200.0,306.0,6.0,0.000652,0.033261,0.518086,1449,0.136456,0.97


In [72]:
df_read[df_read["Quote_date"] <= "2021-12-31"].max()

Unnamed: 0            6526908
Quote_date         2021-12-31
Expire_date        2026-12-18
Underlying_last       4792.94
Strike                 9200.0
Ask                    5059.9
Bid                    4624.9
Bid_strike             46.249
Ask_strike             46.745
Moneyness             47.9294
Ttl                      1873
Volatility           0.543244
R                        2.62
dtype: object

In [66]:
df_read[(df_read["Strike"] == 9200)]

,Unnamed: 0,Quote_date,Expire_date,Underlying_last,Strike,Ask,Bid,Bid_strike,Ask_strike,Moneyness,Ttl,Volatility,R
4784011,6172681,2021-11-05,2024-12-20,4697.16,9200.0,300.0,0.0,0.000000,0.032609,0.510561,1141,0.110251,0.66
4784063,6172733,2021-11-05,2025-12-19,4697.16,9200.0,300.0,0.0,0.000000,0.032609,0.510561,1505,0.110251,1.04
4784088,6172758,2021-11-05,2026-12-18,4697.16,9200.0,309.0,9.0,0.000978,0.033587,0.510561,1869,0.110251,1.04
4793475,6182145,2021-11-08,2024-12-20,4701.65,9200.0,19.2,0.0,0.000000,0.002087,0.511049,1138,0.109971,0.75
4793527,6182197,2021-11-08,2025-12-19,4701.65,9200.0,85.0,5.0,0.000543,0.009239,0.511049,1502,0.109971,1.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5119101,6517400,2021-12-30,2025-12-19,4779.28,9200.0,86.0,6.0,0.000652,0.009348,0.519487,1450,0.137746,0.98
5119161,6517460,2021-12-30,2026-12-18,4779.28,9200.0,43.1,19.1,0.002076,0.004685,0.519487,1814,0.137746,1.27
5128489,6526788,2021-12-31,2024-12-20,4766.39,9200.0,300.0,0.0,0.000000,0.032609,0.518086,1085,0.136456,0.97
5128549,6526848,2021-12-31,2025-12-19,4766.39,9200.0,306.0,6.0,0.000652,0.033261,0.518086,1449,0.136456,0.97
